### Contents
    01. Importing libraries and data
    02. Data wrangling
    03. Consistency and quality check
    04. Reducing data types
    05. Merging the data
    06. Export

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Import Data

In [2]:
# Set path 
path=r'/Users/aylaarreguin/07-15-2020 Instacart Basket Analysis/'

In [3]:
# Import new customer data set as df  
df_cust = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'customers.csv'), index_col = None)

In [4]:
# Check the df 
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_cust.shape

(206209, 10)

# Data Wrangling

In [6]:
# Re-naming columns
df_cust.rename(columns= {'user_id' : 'Customer_id', 'First Name':'First_name', 'Surnam' : 'Surname','STATE' : 'State', 'date_joined': 'Date_joined',
                         'n_dependants':'#_of_dependants', 'fam_status' : 'Marital_status', 'income':'Income'}, inplace = True)

In [7]:
# Check to make sure columns have been successfully renamed 
df_cust.head()

,Customer_id,First_name,Surname,Gender,State,Age,Date_joined,#_of_dependants,Marital_status,Income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# Consistency Checks 

In [8]:
# Check descriptive stats for customer df
df_cust.describe()

,Customer_id,Age,#_of_dependants,Income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [9]:
# Check the shape of the df
df_cust.shape

(206209, 10)

In [10]:
# Check the data types 
df_cust.dtypes

Customer_id         int64
First_name         object
Surname            object
Gender             object
State              object
Age                 int64
Date_joined        object
#_of_dependants     int64
Marital_status     object
Income              int64
dtype: object

In [11]:
# Check for missing values 
df_cust.isnull().sum()

Customer_id            0
First_name         11259
Surname                0
Gender                 0
State                  0
Age                    0
Date_joined            0
#_of_dependants        0
Marital_status         0
Income                 0
dtype: int64

There are 11,259 missing First Name values. Considering I don't have a higher up to contact, and that amount of missing values wont likely affect my analysis, I will leave them. I still have unique User_id values I can use if needed.

In [12]:
# Check for mixed data types 
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

First_name


In [13]:
# Convert first_name data type to string
df_cust['First_name'] = df_cust['First_name'].astype('str')

In [14]:
# Check to see if conversion was successful and there are no other mixed data types 
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

Conversion was successful. There are no mixed data types in the df

In [15]:
# Checking for duplicates
df_cust[df_cust.duplicated()]

,Customer_id,First_name,Surname,Gender,State,Age,Date_joined,#_of_dependants,Marital_status,Income


There are no duplicates in the df

# Reducing data types to save space before merging

In [16]:
df_cust['Age'] = df_cust['Age'].astype('int8')
df_cust['#_of_dependants'] = df_cust['#_of_dependants'].astype('int8')
df_cust['Income'] = df_cust['Income'].astype('int32')
df_cust['Customer_id'] = df_cust['Customer_id'].astype('str')

In [17]:
# Check to see if changes have been successful 
df_cust.dtypes

Customer_id        object
First_name         object
Surname            object
Gender             object
State              object
Age                  int8
Date_joined        object
#_of_dependants      int8
Marital_status     object
Income              int32
dtype: object

# Merge data with ords_prods_merge data

In [18]:
# Import orders_products_merged pkl
ords_prods_merged = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merged_flags.pkl'))

In [19]:
ords_prods_merged.head()

,order_id,customer_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,_merge,Busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_order_price,spending_flag,order_frequency,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,195,...,both,Regularly_busy,Regularly_busy,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,195,...,both,Regularly_busy,Least_busy_days,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,195,...,both,Regularly_busy,Least_busy_days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,195,...,both,Least_busy,Least_busy_days,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,195,...,both,Least_busy,Least_busy_days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [20]:
# Re-naming columns to clean up the look of the df to match the other df (capitalization ect)
ords_prods_merged.rename(columns= {'order_id' : 'Order_id', 'customer_id':'Customer_id', 'order_number' : 'Order_number','orders_day_of_week' : 'Orders_day_of_week', 'order_time_of_day': 'Order_time_of_day','days_since_prior_order':'Days_since_prior_order', 'product_id' : 'Product_id', 'add_to_cart_order':'Add_to_cart_order','reordered':'Reordered','_merge':'_Merge','busiest_days':'Busiest_2_days','busiest_period_of_day':'Busiest_period_of_day','max_order':'Max_order','loyalty_flag':'Loyalty_flag','mean_order_price':'Mean_order_price','spending_flag':'Spending_flag','order_frequency':'Order_frequency','frequency_flag':'Frequency_flag','product_name':'Product_name'}, inplace = True)

In [21]:
# Checking to see if name changes were successful 
ords_prods_merged.head()

,Order_id,Customer_id,Order_number,Orders_day_of_week,Order_time_of_day,Days_since_prior_order,Product_id,Add_to_cart_order,Reordered,Unnamed: 0,...,_Merge,Busiest_day,Busiest_2_days,Busiest_period_of_day,Max_order,Loyalty_flag,Mean_order_price,Spending_flag,Order_frequency,Frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,195,...,both,Regularly_busy,Regularly_busy,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,195,...,both,Regularly_busy,Least_busy_days,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,195,...,both,Regularly_busy,Least_busy_days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,195,...,both,Least_busy,Least_busy_days,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,195,...,both,Least_busy,Least_busy_days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


Everything looks good, however there is one column that can still be deleted. There is no use for the "Unamed" column so I will get rid of it now. 

In [22]:
# Delete "Unnamed" column 
ords_prods_merged = ords_prods_merged.drop(columns = ['Unnamed: 0'])

In [23]:
ords_prods_merged.head()

,Order_id,Customer_id,Order_number,Orders_day_of_week,Order_time_of_day,Days_since_prior_order,Product_id,Add_to_cart_order,Reordered,Product_name,...,_Merge,Busiest_day,Busiest_2_days,Busiest_period_of_day,Max_order,Loyalty_flag,Mean_order_price,Spending_flag,Order_frequency,Frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,both,Regularly_busy,Regularly_busy,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Regularly_busy,Least_busy_days,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Regularly_busy,Least_busy_days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Least_busy,Least_busy_days,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Least_busy,Least_busy_days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [24]:
# Check data types of df ords_prods_merged
ords_prods_merged.dtypes


Order_id                     int32
Customer_id                  int32
Order_number                  int8
Orders_day_of_week            int8
Order_time_of_day             int8
Days_since_prior_order     float16
Product_id                   int32
Add_to_cart_order            int32
Reordered                     int8
Product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_Merge                    category
Busiest_day                 object
Busiest_2_days              object
Busiest_period_of_day       object
Max_order                     int8
Loyalty_flag                object
Mean_order_price           float64
Spending_flag               object
Order_frequency            float16
Frequency_flag              object
dtype: object

In [25]:
# Change the data type of Customer_id to object before merging 
ords_prods_merged['Customer_id'] = ords_prods_merged['Customer_id'].astype('str')

In [26]:
# Merge ords_prods_merged df with the df_cust df 
ords_prods_custs = ords_prods_merged.merge(df_cust, on = 'Customer_id')

In [28]:
ords_prods_custs.head()

,Order_id,Customer_id,Order_number,Orders_day_of_week,Order_time_of_day,Days_since_prior_order,Product_id,Add_to_cart_order,Reordered,Product_name,...,Frequency_flag,First_name,Surname,Gender,State,Age,Date_joined,#_of_dependants,Marital_status,Income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [29]:
# Checking shape.  
ords_prods_custs.shape

(32404859, 32)

In [30]:
ords_prods_custs = ords_prods_merged.merge(df_cust, on = 'Customer_id', how = 'outer', indicator = True)

In [33]:
ords_prods_custs.head()

,Order_id,Customer_id,Order_number,Orders_day_of_week,Order_time_of_day,Days_since_prior_order,Product_id,Add_to_cart_order,Reordered,Product_name,...,First_name,Surname,Gender,State,Age,Date_joined,#_of_dependants,Marital_status,Income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [34]:
ords_prods_custs.shape

(32404859, 33)

In [ ]:
# Export full df as pickle 
ords_prods_custs.to_pickle(os.path.join(path, 'Data','Prepared Data', 'full_instacart_data.pkl'))